In [117]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
from tqdm import tqdm

In [118]:
# Create Model
m = gp.Model("Model_1")

In [119]:
# GENERAL PARAMETERS

# Number of vehicles
V = 6
# Number of nodes i,j
nodes = 4
I = nodes
J = nodes
# Time Steps (days)
T = 11

In [120]:
# EQUATION 5 CONSTRAINTS

# Concurrency Matrix
H = np.matrix("1 1 1 0 ; 0 0 0 1")

# Spacecraft Design Parameters
e = np.matrix("0 0 524 60 500 250 ; 452045 107725 0 18413 8804 2358")

# Create commodity outflow variables and add them to model
def create_add_commodity_outflow(model, V=V, I=I, J=J, T=T):
    variables = {}
    for v, i, j, t in tqdm([(v, i, j, t) for v in range(V) for i in range(I) for j in range(J) for t in range(T)]):
        variables[0, v, i, j, t] = model.addVar(vtype=GRB.CONTINUOUS, name='commodity_outflow_payload')
    for v, i, j, t in tqdm([(v, i, j, t) for v in range(V) for i in range(I) for j in range(J) for t in range(T)]):
        variables[1, v, i, j, t] = model.addVar(vtype=GRB.CONTINUOUS, name='commodity_outflow_crew')
    for v, i, j, t in tqdm([(v, i, j, t) for v in range(V) for i in range(I) for j in range(J) for t in range(T)]):
        variables[2, v, i, j, t] = model.addVar(vtype=GRB.CONTINUOUS, name='commodity_outflow_consumables')
    for v, i, j, t in tqdm([(v, i, j, t) for v in range(V) for i in range(I) for j in range(J) for t in range(T)]):
        variables[3, v, i, j, t] = model.addVar(vtype=GRB.CONTINUOUS, name='commodity_outflow_propellant')
    return variables

commodity_outflow_vector = create_add_commodity_outflow(model=m)

# Create number of spacecraft outflow variables and add them to model
def create_add_number_spacecraft_per_arc(model, V=V, I=I, J=J, T=T):
    variables = {}
    for v, i, j, t in tqdm([(v, i, j, t) for v in range(V) for i in range(I) for j in range(J) for t in range(T)]):
        variables[v, i, j, t] = model.addVar(vtype=GRB.INTEGER, name=f"y{v}{i}{j}{t}")
    return variables

y = create_add_number_spacecraft_per_arc(model=m)

m.update()

100%|██████████| 1056/1056 [00:00<00:00, 479972.37it/s]


In [121]:
# CONSTRAINTS 5

for v, i, j, t in tqdm([(v, i, j, t) for v in range(V) for i in range(I) for j in range(J) for t in range(T)]):
    m.addConstr(commodity_outflow_vector[0,v,i,j,t] + commodity_outflow_vector[1,v,i,j,t] + commodity_outflow_vector[2,v,i,j,t] <= e[0,v] * y[v, i, j, t], "c5_1")
    m.addConstr(commodity_outflow_vector[3,v,i,j,t] <= e[1,v] * y[v, i, j, t], "c5_2")
m.update()

100%|██████████| 1056/1056 [00:00<00:00, 16180.14it/s]


In [131]:
# EQUATION 7 CONSTRAINTS

# Payload Capacity
C = [0, 0, 524, 60, 500, 250]

# Propellant Capacity
M = [452045, 107725, 0, 18413, 8804, 2358]

# Structural Fraction (fuel dependent)
alpha = 0.045   # LOX/kerosene

# Gravitational Acceleration Earth
g_0 = 9.8   # m/s2

# Upper Bound Allowed for Propellant Tank Capacity
M_ub = 500000   # kg

# Specific Impulse
I_sp = [421, 421, 0, 314, 311, 311]

# Spacecraft Impulsive Burn
t_b = 120   # s


# Structure Mass Variable
def create_s_star_variables(model, v=V):
    variables = {}
    for v in range(V):
        variables[v] = model.addVar(vtype=GRB.CONTINUOUS, name=f'Structure_Mass_{v}')
    return variables

s_star = create_s_star_variables(model = m)

m.update()

In [ ]:
# CONSTRAINTS 7

for v in tqdm(V):
    m.addConstr(s_star[v] = 2.3931 * )